<a href="https://colab.research.google.com/github/shabahmd/Machine-Learning-Notebooks/blob/main/BERTSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install and Import Dependencies


In [2]:
!pip install torch transformers requests BeautifulSoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

2. Instantiate Model


In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

3. Encode and Calculate Sentiment


In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great' , return_tensors = 'pt')


In [7]:
results = model(tokens)

In [9]:
results.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [11]:
int(torch.argmax(results.logits)) +1

4

4. Collect Reviews


In [12]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment*')
results = soup.find_all('p', {'class': regex})
reviews = {result.text for result in results }

In [20]:
reviews

{'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.',
 "Great service, lovely location, and really amazing food. Words don't do justice. We had the mushroom parm bruschetta and the loaded double double. Both were so tasty. Also love the Aussie black tea and a flat white. Wish I had more mornings in Sydney to eat breakfast there. Highly recommend.",
 "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.",
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmo

5. Load Reviews into DataFrame and Score


In [15]:
import numpy as np
import pandas as pd

In [21]:
reviews_list = list(reviews)


In [24]:
arr = np.array(reviews_list)

arr_2d = arr.reshape(-1, 1)
df = pd.DataFrame(arr_2d, columns=['review'])


In [25]:
df['review'].iloc[0]

"We came for brunch and they ran out of seven separate menu items. We tried ordering multiple times and each time were let down with them not having what we wanted. It's understandable to not have certain things, but to have half of your menu not available is a bit ridiculous. When we finally were able to get something it was good, but we had to wait around 10-15 minutes after ordering to be told that they were out of something. Disappointing because of high expectations."

In [32]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensor = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +1

In [33]:
sentiment_score(df['review'].iloc[1])


5

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [38]:
df['sentiment'].iloc[7]

5

In [40]:
df.head(5)

,review,sentiment
0,We came for brunch and they ran out of seven s...,2
1,Great place with delicious food and friendly s...,5
2,We came for brunch twice in our week-long visi...,4
3,Very cute coffee shop and restaurant. They hav...,4
4,Ricotta hot cakes! These were so yummy. I ate ...,5
